Importing modules

In [1]:
import pandas as pd
import numpy
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats as sps
import corner

Create data files with data and the headings

In [2]:
data = numpy.loadtxt("Skyserver_Spectro2_13_2018 10_41_13 PM.csv", skiprows = 2, delimiter = ",")

In [3]:
names = numpy.loadtxt("Skyserver_Spectro2_13_2018 10_41_13 PM.csv", dtype=str, skiprows = 1, delimiter = ",")[0]

Convert the text files into a dataframe

In [4]:
original_df = pd.read_csv("/Users/marin/machine/redshift_data_project/Skyserver_SQL6_7_2018 2_06_10 AM.csv", \
                 delimiter = ",", skiprows=1)
original_df.head()

,objid,ra,dec,u,g,r,i,z
0,1237680241978180637,338.968661,-7.091653,24.02065,25.22515,24.07492,25.23330,20.92457
1,1237680241978180409,338.969972,-7.152198,25.86987,23.84455,22.70312,25.44742,22.38291
2,1237680241978180573,338.995865,-7.136117,26.13040,25.10591,25.54131,22.31401,23.76466
3,1237680241978180001,339.013647,-7.122862,22.09485,25.42787,24.04605,24.34716,22.80531
4,1237680241978180643,339.025117,-7.094838,25.02983,24.60557,25.57095,25.15441,20.84509


Filters out values where i = -9999.0, z = -9999.0, and redshift = 0.00

In [5]:
df_filtered = original_df[(original_df['i'] != -9999.0) & (original_df['z'] != -9999.0) & (original_df['redshift'] != 0.00)]
df_filtered.head()

KeyError: 'redshift'

Filters out values where redshift < 0.01, because there are a lot of filler values at 0.01 for some reason

In [ ]:
final_df = df_filtered[(df_filtered['redshift'] > 0.01)]
final_df.head()

Filters out outliers from final_df

In [ ]:
no_outliers_df = final_df[numpy.abs(final_df['redshift']-final_df['redshift'].mean())\
                             <=(3*final_df['redshift'].std())]
no_outliers_df.head()

Creates a new dataframe with four new columns. Each column takes the difference between adjacent columns.

In [ ]:
d= {'u-g': no_outliers_df['u'] - no_outliers_df['g'],
    'g-r': no_outliers_df['g'] - no_outliers_df['r'],
    'r-i': no_outliers_df['r'] - no_outliers_df['i'],
    'i-z': no_outliers_df['i'] - no_outliers_df['z'], 
   }
add_columns_df = pd.DataFrame(d)
add_columns_df.head()

df = pd.concat([no_outliers_df, add_columns_df], axis = 1)

df = df[(df['r'] < 25) & (df['r-i'] > -1) & (df['i-z'] < 2) & (df['i-z'] > -0.5)]
df.head()

Creates x, a dataframe with the columns of magnitude differences and one column of magnitudes (in this case it was u, but it could be anything). Also creates y, which is the redshift column

In [ ]:
y = df['redshift']
x = df.iloc[:,[3, 13, 14, 15, 16]]
x.head()

In [ ]:
x.shape

Defines a manual linear regression function.

In [ ]:
def dumb_linreg(x, coef, intercept):
    return x * coef + intercept

Defines a function which returns 50 floats between the minimum and maximum of a column

In [ ]:
def x_for_plotting(dataframe, column):
    out = numpy.linspace(min(dataframe[column]), max(dataframe[column]), 50)
    return(out)

Defines a chi square error function.

In [ ]:
def chi_square_error(observed_y, predicted_y):
    error = 0
    observed_y_list = observed_y.tolist()
    predicted_y_list = predicted_y.tolist()
    for i in predicted_y_list:
        error += ((observed_y_list[predicted_y_list.index(i)] - i)**2)/numpy.var(predicted_y)**2
    print(error)

In [ ]:
nm_df = x.loc[:, 'u-g':'i-z']

In [ ]:
def y_for_graph(coef,x, i):
    return np.dot(coef[i+1], x)

In [ ]:
nc_x = df.iloc[:,3:8]
nc_x.head()

In [ ]:
nc_y = pd.concat([y]*5, axis=1)
nc_y.head()

# Debugging Plots

a vs d

In [ ]:
plt.scatter(x['u-g'], x['i-z'], color='black', s=1, alpha=0.5)
plt.scatter(x_for_plotting(x, 'u-g'), x_for_plotting(x, 'i-z'), color='blue', s=1, alpha=0.5)
plt.xlabel('u-g')
plt.ylabel('i-z')

In [ ]:
ls = ['u-g', 'g-r', 'r-i', 'i-z']
for i in range(len(ls[:-1])):
    name = ls[i]
    name_2 = ls[i+1]
    plt.scatter(x[name], x[name_2], color='black', s=1, alpha=0.5)
    plt.scatter(x_for_plotting(x, name), x_for_plotting(x, name_2), color='blue', s=1, alpha=0.5)
    plt.xlabel(name)
    plt.ylabel(name_2)
    plt.show()

In [ ]:
for i in range(len(ls[1:-1])):
    name = ls[i]
    name_2 = ls[1:][i+1]
    plt.scatter(x[name], x[name_2], color='black', s=1, alpha=0.5)
    plt.scatter(x_for_plotting(x, name), x_for_plotting(x, name_2), color='blue', s=1, alpha=0.5)
    plt.xlabel(name)
    plt.ylabel(name_2)
    plt.show()

# Why The Algorithm Doesn't Work

$y_o$ = observed y values
$y_p$ = predicted y values

$y = mx + b$ is an equation where y and b are scalars, and m and x are vectors.

Function $f$ $\exists$ where $x_2 = f(x_1)$ 

Function $g$ $\exists$ where $x_3 = g(x_2)$

Our equation is
$y = m \cdot \begin{bmatrix}x_1,  f(x_1),  g(f(x_1))\end{bmatrix} + b$

The equations we want to solve are as follows:
$y_p - m_1 \cdot x_1 + b$ 
$y_p - m_2 \cdot f(x_1) + b$
$y_p - m_3 \cdot g(f(x_1)) + b$

But when multiplying
$y = \begin{bmatrix} m_1, m_2, m_3\end{bmatrix} \cdot \begin{bmatrix} x_1, x_2, x_3\end{bmatrix}$

we instead got
$y_p = m_1 \cdot x_1 + m_2 \cdot x_2 + m_3 \cdot x_3 + b$

Since the two results aren't the same, the lines look really strange.

$x_{o,n}$ = observed x values
$x_{p,n}$ = x values for plotting

Relationships between $x_1, x_2, x_3$:

$x_{p,2} = 0.55915161 \cdot x_{p,1}$

$x_{p,3} = 0.52240939 \cdot x_{p,2}$

$x_{o,2} = 0.48012458 \cdot x_{o,1}$

$x_{o,3} = 0.54765443 \cdot x_{o,2}$

$x_{o,4} = 4.00706762 \cdot x_{o,3}$

# Calculations for Desired Equations



Set of slopes ($m_n$): $[0.40333345,  0.04263828, -0.0342513,  0.00849987]$

Intercept ($b$): 0.058753418732004334

$x_{p,1}$

In [ ]:
x_for_plotting(x, 'u-g')

$x_{p,2}$ = $f(x_1)$

In [ ]:
x_for_plotting(x, 'u-g')

$y_p = m_1 \cdot x_{p,1} + b$

In [ ]:
np.dot(0.40333345, x_for_plotting(x, 'u-g')) + 0.058753418732004334

$y_p = m_2 \cdot x_{p,2} + b $

$y_p = m_2 \cdot f(x_{p,1}) + b $

In [ ]:
np.dot(0.04263828, x_for_plotting(x, 'u-g') * 0.55915161) + 0.058753418732004334

$y_p = m_3 \cdot x_{p,3} + b $ 

$y_p = m_3 \cdot g(f(x_{p,1})) + b $

In [ ]:
np.dot(-0.0342513, 0.52240939 * (0.55915161 * x_for_plotting(x, 'u-g'))) + 0.058753418732004334

Equation for $y_p = m_1 \cdot x_1 + m_2 \cdot x_2 + m_3 \cdot x_3 + b$ using actual values for plotting

In [ ]:
y_p1 = 0.40333345 * x_for_plotting(x, 'u-g') + 0.04263828 * x_for_plotting(x, 'g-r') \
-0.0342513 * x_for_plotting(x, 'r-i') + 0.058753418732004334 

Equation for $y_p = m_1 \cdot x_1 + m_2 \cdot x_2 + m_3 \cdot x_3 + b$ using $f(x_1)$ and $g(f(x_1))$

In [ ]:
y_p2 = 0.40333345 * x_for_plotting(x, 'u-g') + 0.04263828 * (0.55915161 * x_for_plotting(x, 'u-g')) + \
(0.52240939 * (0.55915161 * x_for_plotting(x, 'u-g'))) + 0.058753418732004334 

In [ ]:
plt.plot(x_for_plotting(x, 'u-g'), y_p2, color='blue', label='plot using functions')
plt.plot(x_for_plotting(x, 'u-g'), y_p1, color='orange', label='plot using actual values')
legend = plt.legend(loc='upper left', shadow=True, fontsize='x-large')
plt.xlabel('x_p1 values')
plt.ylabel('predicted y values')
plt.show()

# Different Algorithms

In [ ]:
corner.corner(x)

In [ ]:
def density_estimation(m1, m2):
    X, Y = np.mgrid[min(m1):max(m1):100j, min(m2):max(m2):100j]                                                     
    positions = np.vstack([X.ravel(), Y.ravel()])                                                       
    values = np.vstack([m1, m2])                                                                        
    kernel = sps.gaussian_kde(values)                                                             
    Z = np.reshape(kernel(positions).T, X.shape)
    return X, Y, Z


In [ ]:
def mycorner(data, keys, colors, maps, lims=None, pre_densities=None, filename='plot.pdf'):
    ncol = len(keys)
    fig = plt.figure(figsize=(ncol*5, ncol*5))
    ax = [[fig.add_subplot(ncol, ncol, ncol * i + j + 1) for j in range(i+1)] for i in range(ncol)]
    for k in range(len(data)):
        datum = data[k]
        npoints = len(datum)
        for i in range(ncol):
            for j in range(i+1):
                if i == j:
                    ax[i][j].hist(datum[keys[i]], bins=50, histtype='step', linewidth=2, normed=True, alpha=0.75, color=colors[k])
                    ax[i][j].set_xlabel(keys[i])
                else:
                    if npoints > 1e4:
                        ax[i][j].hist2d(datum[keys[i]], datum[keys[j]], bins=(100, 100), normed=True, cmap=maps[k], alpha=0.5)
                    else:
                        if pre_densities is None:
                            x, y, z = density_estimation(datum[keys[i]], datum[keys[j]])
                        else:
                            (x, y, z) = pre_densities[i][j]
                        ax[i][j].contour(x, y, z, cmap=plt.get_cmap(maps[k]) , alpha=0.5)
                    ax[i][j].set_xlabel(keys[i])
                    ax[i][j].set_ylabel(keys[j])
                    if lims is not None:
                        ax[i][j].set_xlim(lims)
                        ax[i][j].set_ylim(lims)
    fig.savefig(filename, dpi=100)
    return

In [ ]:
mycorner([x], ['u-g', 'g-r', 'r-i', 'i-z'], ['r'], ['Reds'], filename='big_corner_coarse.png')